- -> exp037
- transformer 試す

In [1]:
%load_ext autoreload

In [2]:
import os

is_kaggle_notebook = os.path.exists("/kaggle/input")

# 必要パッケージをインストール
if is_kaggle_notebook:
    !pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
    !pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl

In [3]:
import os
import sys
import warnings
from pathlib import Path

import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
import wandb
from rdkit import rdBase
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GroupKFold, KFold
from tqdm.auto import tqdm

rdBase.DisableLog("rdApp.warning")
warnings.filterwarnings("ignore")

In [9]:
pr_number = 1
debug = True
augmented_feature = '["morgan", "maccs"]'
graph_pooling = "mean"
notes = ""
is_trimmer_cyclic = False
exp = "exp040"
batch_size = 256
num_epochs = 1000
drop_ratio = 0.5
set_seed = True

In [5]:
if set_seed:
    SEED = 42
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    g = torch.Generator()
    g.manual_seed(SEED)

    # # 決定論モード
    # torch.use_deterministic_algorithms(True)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
    

In [6]:
%autoreload 2

if is_kaggle_notebook:
    module_path = f"/kaggle/input/myproject-pr-{pr_number:04}"
    !mkdir src
    !cp -r $module_path/* src/
    src_path = "./"
else:
    src_path = "../"

sys.path.append(src_path)

from src.data import add_descriptors, add_external_data, load_data, make_smile_canonical, generate_trimer
from src.model import get_model
from src.utils import NULL_FOR_SUBMISSION, generate_scaffold, score, add_scaffold_kfold, scaffold_cv_split
from src.utils.upload_kaggle_dataset import (
    create_kaggle_dataset_metadata,
    upload_kaggle_dataset,
)

wandb.login()

wandb: Currently logged in as: ko_ya346 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
model_names = [
    # "gnn",
    # "bfgnn", "dir", "grin", "irm", "lstm", "rpgnn", 
    "smiles_transformer", 
    # "ssr"
]
config = {
    "debug": debug,
    "n_splits": 5,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "drop_ratio": drop_ratio,
    "graph_pooling": graph_pooling,
    "is_trimmer_cyclic": is_trimmer_cyclic,
}
smiles_col = "SMILES_trimmer_cyclic" if is_trimmer_cyclic else "SMILES"

data_exp = "exp040"

dataset_title = f"model-{exp}"
dataset_id = f"koya346/{dataset_title}"

if is_kaggle_notebook:
    config["debug"] = False

if config["debug"]:
    config["n_splits"] = 2
    config["num_epochs"] = 10

targets = ["Tg", "FFV", "Tc", "Density", "Rg"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if isinstance(augmented_feature, str):
    augmented_feature = eval(augmented_feature)

cuda


In [20]:
# ---------------------------
# メイン処理
# ---------------------------
if config["debug"]:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp / "debug"
    data_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp / "debug"
else:
    output_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / exp

model_output_path = output_path / "model"
if not os.path.exists(model_output_path):
    os.makedirs(model_output_path)

if is_kaggle_notebook:
    # kaggle notebook
    data_dir = Path("/kaggle/input")
else:
    # local
    data_dir = Path("/home/kouya-takahashi/kaggle/opp2025/data/raw")

if config["debug"]:
    train_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / data_exp / "train_debug.csv"
else:
    train_path = Path("/home/kouya-takahashi/kaggle/opp2025/outputs") / data_exp / "train.csv"

    
if os.path.exists(train_path):
    print("Load train.csv")
    
    train = pd.read_csv(train_path)
else:
    train, _ = load_data(data_dir)
    
    if config["debug"]:
        # 各ターゲットが欠損していないデータを30 件ずつ取り出す
        tmp_dfs = []
        for target in targets:
            cond = train[target].notnull()
            tmp_dfs.append(train[cond].iloc[:30])
        train = pd.concat(tmp_dfs).reset_index(drop=True)
    else:
        print(train.shape)
        external_data_dict = [
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv",
                "col": "Tc",
                "rename_d": {"TC_mean": "Tc"},
            },
            {
                "ex_path": data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv",
                "col": "FFV",
            },
            {
                "ex_path": data_dir / "tg-smiles-pid-polymer-class/TgSS_enriched_cleaned.csv",
                "col": "Tg",
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_dnst1.xlsx",
                "col": "Density",
                "rename_d": {"density(g/cm3)": "Density"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/data_tg3.xlsx",
                "col": "Tg",
                "rename_d": {"Tg [K]": "Tg"}, 
            },
            {
                "ex_path": data_dir / "smiles-extra-data/JCIM_sup_bigsmiles.csv",
                "col": "Tg",
                "rename_d": {"Tg (C)": "Tg"}, 
            },
        ]
        for d in external_data_dict:
            print(f"ex_path: {str(d['ex_path'])}")
            train = add_external_data(
                df=train,
                ex_path=d.get("ex_path"),
                col=d.get("col"),
                rename_d=d.get("rename_d"),
                is_complement=config["is_complement"]
            )
            print("after train.shape: ", train.shape)
    
    # トリマー環状化合物を生成
    if config["is_trimmer_cyclic"]:
        error_cnt = 0
        trimer_smiles = []
        for smiles in tqdm(train["SMILES"].values):
            try:
                trimer_smiles.append(generate_trimer(smiles))
            except ValueError as e:
                print(f"smiles: {smiles}, {e}")
                trimer_smiles.append(smiles)
                error_cnt += 1
        train[smiles_col] = trimer_smiles
        print(f"error smiles count: {error_cnt}")
    
    train["SMILES"] = train["SMILES"].apply(make_smile_canonical)
    train["scaffold"] = train["SMILES"].apply(generate_scaffold)
    
    # id が欠損している -> 追加データ
    train["is_external"] = train["id"].isnull()
    
    train["id"] = np.arange(len(train))
    print("Save train.csv")
    train.to_csv(train_path, index=False)


submission = pd.read_csv(
    data_dir / "neurips-open-polymer-prediction-2025/sample_submission.csv"
)


fold_results = []
train_config = {
    "task_type": "regression",
    "num_task": 1,
    "batch_size": config["batch_size"],
    "augmented_feature": augmented_feature,
    "epochs": config["num_epochs"],
    "drop_ratio": config["drop_ratio"],
    "patience": 50,
    "scheduler_patience": 5,
    "verbose": False,
    "graph_pooling": config["graph_pooling"]
}

all_loss_tables = []
for model_name in model_names:
    train_model_config = train_config.copy()

    # モデルによって引数変わる
    if model_name == "smiles_transformer":
        del train_model_config["augmented_feature"], train_model_config["drop_ratio"], train_model_config["graph_pooling"]
    
    wandb_name = f"{exp}_{model_name}" if not config["debug"] else f"{exp}_{model_name}_debug"
    wandb.init(project="opp2025", name=f"{exp}_{model_name}", config=config)
    wandb.log({"Notes": notes})
    loss_table = wandb.Table(columns=["exp", "model_name", "fold", "target", "mae", "mse"])
    oof_dfs = []
    mae_dict = {}
    
    print("====================")
    print(f"model_name: {model_name}")

    for target in targets:
        cond = train[target].notnull()
        df_train = train[cond]
        df_train_org = df_train[~df_train["is_external"]].reset_index(drop=True)
        df_train_external = df_train[df_train["is_external"]].reset_index(drop=True)
        
        df_train_idx = df_train_org["id"].values
        X_org = df_train_org[[smiles_col]]
        y_org = df_train_org[target]
        X_external = df_train_external[[smiles_col]]
        y_external = df_train_external[target]
        oof = np.zeros(len(X_org))

        df_train_org = add_scaffold_kfold(df_train_org, n_splits=config["n_splits"])
        
        for fold, train_idx, val_idx in scaffold_cv_split(df_train_org, n_splits=config["n_splits"]):
            print(f'\n=== Fold {fold + 1} / {config["n_splits"]} ===')

            X_train = pd.concat([X_org.iloc[train_idx][smiles_col], X_external[smiles_col]]).to_list()
            y_train = pd.concat([y_org.iloc[train_idx], y_external]).to_numpy()
            X_val = X_org.iloc[val_idx][smiles_col].to_list()
            y_val = y_org.iloc[val_idx].to_numpy()
    
            model = get_model(model_name)(**train_model_config)
            #     task_type="regression",
            #     num_task=1,
            #     batch_size=config["batch_size"],
            #     # augmented_feature=augmented_feature,
            #     epochs=config["num_epochs"],
            #     # drop_ratio=config["drop_ratio"],
            #     patience=50,
            #     scheduler_patience=5,
            #     verbose=False,
            #     # graph_pooling=config["graph_pooling"]
            # )
            model.fit(
                # gnn.autofit(
                X_train=X_train,
                y_train=y_train,
                X_val=X_val,
                y_val=y_val,
                # n_trials=1,
            )
            save_model_path = model_output_path / f"{model_name}_{fold}_{target}.pt"
            model.save(path=str(save_model_path))
    
            preds = model.predict(X_val)["prediction"].flatten()
            oof[val_idx] = preds
            
            mse = mean_squared_error(y_val, preds)
            mae = mean_absolute_error(y_val, preds)
            print(f"Fold {fold+1} target: {target} MSE: {mse:.4f} | MAE: {mae:.4f}")
            loss_table.add_data(exp, model_name, fold, target, mae, mse)

            all_loss_tables.append({"target": target, "fold": fold, "mae": mae})
            
        score_mse = mean_squared_error(y_org, oof)
        score_mae = mean_absolute_error(y_org, oof)
        print(f"RMSE for {target}: {score_mse:.4f}")
        print(f"MAE for {target}: {score_mae:.4f}")
        mae_dict[target] = score_mae

        oof_df = pd.DataFrame({
            "id": df_train_idx,
            target: oof            
        })
        oof_dfs.append(oof_df)

        wandb.log({f"fold_target_losses": loss_table})

    # CV 計算
    oof_df = pd.DataFrame()
    oof_df["id"] = train.loc[~train["is_external"], "id"]
    for i_oof in oof_dfs:
        oof_df = oof_df.merge(i_oof, on="id", how="left")
    solution = train.loc[~train["is_external"], ["id"] + targets].copy()
    solution = solution.fillna(NULL_FOR_SUBMISSION)
    oof_df = oof_df.fillna(NULL_FOR_SUBMISSION)
    
    # 評価
    final_score = score(
        solution=solution,
        submission=oof_df,
    )
    print(f"\n📊 Final OOF Score (wMAE): {final_score:.6f}")
    wandb.log({"wMAE": final_score})

    oof_df.to_csv(output_path / f"oof_{model_name}.csv", index=False)

    
    # target 毎の 平均 mae を記録
    for target in targets:
        key_name = f"{target}_mean_mae"
        mae_values = mae_dict[target]
        # mae_values = [d["mae"] for d in all_loss_tables if d["target"] == target]
        wandb.log({key_name: np.mean(mae_values)})


Load train.csv


Density_mean_mae,▁
FFV_mean_mae,▁
Rg_mean_mae,▁
Tc_mean_mae,▁
Tg_mean_mae,▁
wMAE,▁
Density_mean_mae,0.93931
FFV_mean_mae,0.35013
Notes,
Rg_mean_mae,16.33129
Tc_mean_mae,0.23786


model_name: smiles_transformer

=== Fold 1 / 2 ===
Fold 1 target: Tg MSE: 8267.8663 | MAE: 72.6873

=== Fold 2 / 2 ===
Fold 2 target: Tg MSE: 15149.5800 | MAE: 83.1588
RMSE for Tg: 11708.7231
MAE for Tg: 77.9230

=== Fold 1 / 2 ===
Fold 1 target: FFV MSE: 0.1247 | MAE: 0.3522

=== Fold 2 / 2 ===
Fold 2 target: FFV MSE: 0.1217 | MAE: 0.3479
RMSE for FFV: 0.1232
MAE for FFV: 0.3501

=== Fold 1 / 2 ===
Fold 1 target: Tc MSE: 0.0726 | MAE: 0.2467

=== Fold 2 / 2 ===
Fold 2 target: Tc MSE: 0.0615 | MAE: 0.2268
RMSE for Tc: 0.0671
MAE for Tc: 0.2369

=== Fold 1 / 2 ===
Fold 1 target: Density MSE: 0.8345 | MAE: 0.9038

=== Fold 2 / 2 ===
Fold 2 target: Density MSE: 0.9528 | MAE: 0.9697
RMSE for Density: 0.8944
MAE for Density: 0.9371

=== Fold 1 / 2 ===
Fold 1 target: Rg MSE: 285.9472 | MAE: 16.0584

=== Fold 2 / 2 ===
Fold 2 target: Rg MSE: 310.3715 | MAE: 16.6146
RMSE for Rg: 298.0086
MAE for Rg: 16.3331

📊 Final OOF Score (wMAE): 0.517654


In [ ]:
if not config["debug"]:
    create_kaggle_dataset_metadata(dataset_title, dataset_id, model_output_path)
    upload_kaggle_dataset(dataset_id, model_output_path)

In [19]:


# dict をコピーしてキー削除したい
d = {"a": 1, "b": 2, "c": 3}
dd = d.copy()

del dd["a"]
print(d, dd)

{'a': 1, 'b': 2, 'c': 3} {'b': 2, 'c': 3}
